In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [2]:
path = "/Users/denniscimorosi/Desktop/Tesi/IPinYou/make-ipinyou-data-master/all/"
test_name = "test.log.txt"
train_name = "train.log.txt"

In [3]:
train_df = pd.read_csv(path + train_name, delimiter='\t')
test_df = pd.read_csv(path + test_name, delimiter='\t')

/var/folders/0w/0z7x_4h92p7cfg3jdw8y5jg80000gn/T/ipykernel_1127/996104040.py:1: DtypeWarning: Columns (15,18,19,21,24) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(path + train_name, delimiter='\t')
/var/folders/0w/0z7x_4h92p7cfg3jdw8y5jg80000gn/T/ipykernel_1127/996104040.py:2: DtypeWarning: Columns (15,18,19,21,24) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv(path + test_name, delimiter='\t')


In [4]:
upsample = False
encoding = 'one-hot'

In [5]:
train_df = train_df.drop(['bidid', 'logtype', 'ipinyouid', 'IP', 'adexchange',
                          'urlid', 'url', 'slotid', 'slotwidth', 'slotheight', 'slotvisibility',
                          'slotformat', 'creative', 'keypage','usertag'], axis=1)

test_df = test_df.drop(['bidid', 'logtype', 'ipinyouid', 'IP', 'adexchange',
                          'urlid', 'url', 'slotid', 'slotwidth', 'slotheight', 'slotvisibility',
                          'slotformat', 'creative', 'keypage', 'usertag', 'nclick', 'nconversation'], axis=1)

In [6]:
test_df['click'].sum()

3008

In [7]:
train_df.shape[0]

15395258

In [8]:
# split useragent columns into os and browser 
def split_os_browser(df):
    def extract_os(x):
        return x.split("_")[0]

    def extract_browser(x):
        return x.split("_")[1]

    vfun = np.vectorize(extract_os)
    df['os'] = vfun(df['useragent'])

    vfun = np.vectorize(extract_browser)
    df['browser'] = vfun(df['useragent'])

    df.drop(['useragent'], inplace=True, axis=1)

    return df

train_df = split_os_browser(train_df)
test_df = split_os_browser(test_df)

In [9]:
#df = train_df.groupby('city', as_index=False).count().sort_values(['click'])

In [10]:
#df[df['click'] > 170000].shape[0]

In [11]:
# nan domains handling
train_df['domain'] = train_df['domain'].fillna('unknown')
test_df['domain'] = test_df['domain'].fillna('unknown')

def aggregate(x):
    return 'other' if x not in to_keep else str(x)

# aggregate domains which occur less than 100000 times (tot 18)
domains = train_df.groupby('domain', as_index=False).count()
to_keep = domains[domains['click'] > 110000]['domain']
to_keep = np.array(to_keep)

vfun = np.vectorize(aggregate)
train_df['domain'] = vfun(train_df['domain'])
test_df['domain'] = vfun(test_df['domain'])

# aggregate cities which occur less than 120000 times (tot 16)
cities = train_df.groupby('city', as_index=False).count()
to_keep = cities[cities['click'] > 170000]['city']
to_keep = np.array(to_keep)

vfun = np.vectorize(aggregate)
train_df['city'] = vfun(train_df['city'])
test_df['city'] = vfun(test_df['city']) 

# aggregate regions which occur less than 500000 times (tot 10)
regions = train_df.groupby('region', as_index=False).count()
to_keep = regions[regions['click'] > 500000]['region']
to_keep = np.array(to_keep)

vfun = np.vectorize(aggregate)
train_df['region'] = vfun(train_df['region'])
test_df['region'] = vfun(test_df['region'])

In [12]:
# label encoding
def label_enc(test_df, train_df, feature_name):
    # extract all feature values from both train and test df
    test_feats = test_df[feature_name].unique()
    train_feats = train_df[feature_name].unique()
    all_feats = np.unique(np.concatenate([train_feats, test_feats]))
    # fit label encoder
    encoder = preprocessing.LabelEncoder().fit(all_feats)
    # transform feature
    test_df[feature_name] = encoder.transform(test_df[feature_name])
    train_df[feature_name] = encoder.transform(train_df[feature_name])
    return test_df, train_df

if encoding == 'label':
    test_df, train_df = label_enc(test_df, train_df, 'os')
    test_df, train_df = label_enc(test_df, train_df, 'browser')
    test_df, train_df = label_enc(test_df, train_df, 'domain')

In [26]:
def one_hot_enc(test_df, train_df, feature_name):
    test_cols = []
    train_cols = []

    # extract all feature values from both train and test df
    test_feats = test_df[feature_name].unique()
    train_feats = train_df[feature_name].unique()
    all_feats = np.unique(np.concatenate([train_feats, test_feats]))

    # fit one-hot encoder
    encoder = preprocessing.LabelBinarizer().fit(all_feats.reshape(-1, 1))

    
    return encoder, test_df, train_df
    
    '''
    transformed = encoder.transform(test_df[feature_name])
    test_ohe = pd.DataFrame(transformed)

    transformed = encoder.transform(train_df[feature_name])
    train_ohe = pd.DataFrame(transformed)

    for col in test_ohe.columns:
        test_cols.append(feature_name + '_' + str(col))
    test_ohe.columns = test_cols
    test_df = test_df.join(test_ohe).drop([feature_name], axis=1)

    for col in train_ohe.columns:
        train_cols.append(feature_name + '_' + str(col))
    train_ohe.columns = train_cols
    train_df = train_df.join(train_ohe).drop([feature_name], axis=1)

    return test_df, train_df
    '''
    
if encoding == 'one-hot':
    encoder, test_df, train_df = one_hot_enc(test_df, train_df, 'advertiser')
    '''
    test_df, train_df = one_hot_enc(test_df, train_df, 'os')
    test_df, train_df = one_hot_enc(test_df, train_df, 'browser')
    test_df, train_df = one_hot_enc(test_df, train_df, 'domain')
    test_df, train_df = one_hot_enc(test_df, train_df, 'city')
    test_df, train_df = one_hot_enc(test_df, train_df, 'region')
    '''


In [32]:
encoder.inverse_transform(np.array([[1,0,0,0,0,0,0,0,0]]))

array([1458])

In [ ]:
train_df.to_csv(path + 'train.final.csv', index = False)
test_df.to_csv(path + 'test.final.csv', index = False)

In [ ]:
train_df.columns